# Single-particle motion workbook

This notebook provides an investigation into single-particle motion within a laser.  It was developed by Kyle Miller based on notes from Professor W.B. Mori's graduate class in which exact analytical solutions of the particle motion are obtained. An improvement for the expansion in these notes was found by Kyle Miller and is presented here. Some of the calculations have also been presented by J. Yang, R. Craxton, and M. Haines, "Explicit general solutions to relativistic electron dynamics in plane-wave electromagnetic fields and simulations of ponderomotive acceleration," Plasma Phys. Contr. Fusion **53**, 125006 (2011).  This notebook can also be used to explore the accuracy of the equation of motion (Boris push and extensions) used in particle-in-cell (PIC) simulations for the motion of a single-particle in the analytical fields of a plane wave.   Parameters such as the laser intensity as well as initial phase can be varied to compare results against theory.

Consider a plane wave and linearly polarized laser with a profile

$\vec{A}=\hat{x} A(\xi) \cos (k_0 \xi+\phi), \qquad \vec{E} = \hat{x}\left[ -\frac{dA}{d\xi} \cos (k_0 \xi+\phi) + A(\xi) k_0 \sin (k_0 \xi+\phi) \right], \qquad \vec{k}=\hat{z}k_0$.

where $\xi\equiv z-ct$. In class we showed that for a single particle interacting with such a wave two quanities are conserved: $\gamma-p_z/mc$ and $p_x+qA/c$ (the canonical momemtum in the translationally invariant direction).  If we initialize a particle in a laser with constant amplitude $A(\xi)=A_0$ and with zero initial velocity at $x=z=0$ and $\phi=\pi/2$ (or any $\phi=(4n+1)\pi/2)$, then we showed that you can express the longitudinal motion $z(t)$ as a drift plus some oscillation.  An explicit solution for this was given in the class notes as

$z(t) = v_d t + \frac{1}{k_0} \sum_{n=1}^{\infty} \frac{(-1)^{n}}{n!} J_n\left(\frac{n v_d}{c}\right) \sin\left(2n\Omega_0t\right)$,

where $v_d = \frac{c A_0^2}{4+A_0^2}$ and $\Omega_0=k_0 c\left( 1-\frac{v_d}{c} \right)$.  One can also obtain explicit time solutions for $x$, $p_z$, and $p_x$, which are not given here. Following the procedures in the notes,   one can extend the theory to include cases where particles do not start at rest and/or start at different phases of the laser. For the rest of this workbook, analytical solutions will be taken from Yang et al., which allow for general initial conditions of the particle and phase of the laser.

Below you can run OSIRIS simulations in which a laser propagates towards a particle that is either starting at rest or already in motion. The trajectories of various quantities for the particles are plotted along with theory curves. You can choose your own parameters for the laser amplitude ($A_0$), initial phase deviation from $\pi/2$ (could be thought of as $\delta \phi$; in input deck below $\phi$ is given in *units of degrees*), and initial longitudinal proper velocity of the particle ($u_{z0}=\gamma v_{z0}$).  You will also be able to run with various field solvers and particle pushers.  The laser profile rises rapidly, is flat over a large region, and then falls rapidly. The rise and fall taper off smoothly to zero.   This is necessary to avoid unphysical results from the rise and fall of the laser.

The two field solvers available are the Yee and Fei solvers.  The Yee solver is a standard among PIC codes, but has an imperfect dispersion relation for light waves in vacuum.  The Fei (yes, he is your TA) solver uses a 16-point stencil in the longitudinal direction to correct for dispersion errors in solving Maxwell's equations as well as for offset electric and magnetic fields in time.

The six particle pushers are Boris (`standard`), Vay (`vay`), conditional Vay (`cond_vay`), Cary (`cary`), full rotation (`fullrot`), and Euler (`euler`).  The standard Boris push is the most common used in PIC codes.  The Vay pusher preserves the $\vec{E}\times\vec{B}$ velocity but is not volume-preserving, and is useful when particles attain very high energies (for large values of $A_0$); it can also lead to the magnetic field doing work.  The conditional Vay pusher uses the standard Boris push for particles with $\gamma<5$ and the Vay pusher otherwise.  The Cary pusher preserves the $\vec{E}\times\vec{B}$ velocity like the Vay pusher, but it is also volume-preserving, like the standard Boris push.  The full rotation and Euler pushers are like the standard Boris push but perform the $\vec B$ field rotation exactly, and should be identical for our purposes.

For this notebook, we like to experiment to understand the theoretical trajectories, gain confidence in the PIC codes, and to understand some limitations.

Several examples that have proved insightful include the following:

* Yee, $t_f=40$, $u_{z0}=0$, $a_0=0.5$, standard pusher.
    * Do other pushers make much of a difference?
    * Does the Fei solver make much of a difference?
* Yee, $t_f=300$, $u_{z0}=0$, $a_0=5$
    * Try other pushers.  Is any one significantly better?  Significantly worse?
    * Try halving $dt$. Can you get it to converge?  Think about the dispersion relation using the Yee solver as you do this.
    * Does the Fei solver make much of a difference?
* Change $u_{z0}=3$ in the above example and repeat.  **This makes a big difference!**
    * Which pushers give the right answer when using $dt=1$ with the Fei solver?
* Yee/Fei, $t_f=600$, $u_{z0}=20$, $a_0=50$, standard, Vay, Cary, and full rotation pushers
    * The trends in this example are all amplified compared to the other cases.  How important is the Fei solver for this case?  What about the different pushers?
    * Try reducing $dt$ for the Yee solver with Cary pusher.  Can you get as good agreement for any time step value as you can when using the Fei solver and Cary pusher with $dt=1$?  Think about the dispersion relation for the Yee solver with a small time step.

OSIRIS simulations are done in normalized units. 

* Time:  $t' = t \omega_{p}$

* Frequency: $\omega' = \frac{\omega}{\omega_{p}}$ 

* Position: $\vec{x}' = \frac{\omega_{p}}{c} \vec{x}$  

* Momenta: $\vec{u}' = \frac{\vec{p}}{m_{e} c} = \frac{\vec{\gamma v}}{c} = \frac{\vec{u}} {c}$  

* Electric field: $\vec{E}' = e \frac{c / \omega_{p}}{m_{e} c^2} \vec{E}$  

* Magnetic field: $\vec{B}' = e \frac{c / \omega_{p}}{m_{e} c^2} \vec{B}$  


In [ ]:
# **********************************
# First Run This Cell to 
# import the necessary libraries
# **********************************
#
# Please run this cell before running anything, and run this cell again if you have restarted the 
# python kernel.
#
# This cell imports useful routines to diagnose the simulations here
#

import h5py
%matplotlib inline
import numpy as np
import sys, os
import osiris
import matplotlib.pyplot as plt
from single_particle_helper import single_particle_widget, newifile, plot_data
from ipywidgets import interact_manual,Layout,interact, FloatSlider
import ipywidgets as widgets
interact_calc=interact_manual.options(manual_name="Make New Input and Run")

### Simulation Sandbox --- Make your own simulation

Execute the cell below only once, which will open up a widget.  Enter your parameters into the widget, including a name for the output. When the button is clicked, the function will make an input deck for OSIRIS and run the simulation.  After the simulation has completed, execute the next cell, which will plot the data along with the exact analytical solution.  The name you chose is automatically carried over into the plotting routine, but feel free to change `fldr_name` to another value if you wish to visualize other data.  You can then edit the parameters, click the button to run, and execute the diagnostic cell repeatedly.  The following parameters are available to adjust in the `plot_data` function:
* offset: if you wish to adjust the phase of the theory curve plotted
* theory: set to False if you don't want the theory curve plotted
* xlim_max: Customize the maximum value shown on the x-axis.  Otherwise it will plot the most data available
* plot_z: set to True if you want to plot $z$ instead of $\xi$
* save_fig: set to False if you don't want the figure to automatically save to file

In [ ]:
## Hit <shift>+<return> to start the widget, which contains a interface to change OSIRIS parameters

name=single_particle_widget(run_osiris=True)

In [ ]:
## Hit <shift>+<return> to plot the data along with the theory

fldr_name = name.value
print('Plotting data for '+fldr_name)
plot_data(fldr_name,offset=None,theory=True,xlim_max=None,plot_z=False,save_fig=True)